In [1]:
import pandas as pd
import json
import os
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium import webdriver
import csv
import re
from urllib.request import urlretrieve
import requests

In [2]:
df = pd.read_csv("scraper_list/google_list_v2.csv")

In [3]:
print(len(df))

19502


In [4]:
df.head()

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link
0,US-11604547-B2,Multipoint touchscreen,Apple Inc.,"Steve Hotelling, Joshua A. Strickon, Brian Q. ...",2004-05-06,2021-01-27,2023-03-14,2023-03-14,https://patents.google.com/patent/US11604547B2/en,https://patentimages.storage.googleapis.com/a1...
1,US-11029838-B2,"Touch screen device, method, and graphical use...",Apple Inc.,"Stephen O. Lemay, Richard Williamson",2006-09-06,2019-12-04,2021-06-08,2021-06-08,https://patents.google.com/patent/US11029838B2/en,https://patentimages.storage.googleapis.com/bf...
2,US-11379541-B2,System and method for adapting a control funct...,Autoconnect Holdings Llc,Christopher P. Ricci,2013-04-15,2021-04-16,2022-07-05,2022-07-05,https://patents.google.com/patent/US11379541B2/en,https://patentimages.storage.googleapis.com/f0...
3,US-11390881-B2,Soybean event MON89788 and methods for detecti...,"Monsanto Technology, Llc","Marianne Malven, Jennifer Rinehart, Nancy Tayl...",2005-05-27,2020-07-01,2022-07-19,2022-07-19,https://patents.google.com/patent/US11390881B2/en,https://patentimages.storage.googleapis.com/3f...
4,US-11423886-B2,Task flow identification based on user intent,Apple Inc.,"Thomas Robert Gruber, Adam John Cheyer, Dag Ki...",2010-01-18,2020-05-20,2022-08-23,2022-08-23,https://patents.google.com/patent/US11423886B2/en,https://patentimages.storage.googleapis.com/6a...


In [5]:
df.columns

Index(['id', 'title', 'assignee', 'inventor/author', 'priority date',
       'filing/creation date', 'publication date', 'grant date', 'result link',
       'representative figure link'],
      dtype='object')

In [6]:
chrome_options = ChromeOptions()
chrome_options.add_argument("--headless")
driver=webdriver.Chrome()

In [7]:
def test_selenium(url):
    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(5) 
    # pubnum = driver.find_element(By.CSS_SELECTOR, "h2#pubnum")
    # title = driver.find_element(By.CSS_SELECTOR, "h1#title")
    # abstract = driver.find_element(By.CSS_SELECTOR, ".abstract.patent-text")
    # claims_div = driver.find_element(By.CSS_SELECTOR, ".claims.patent-text")
    # claims = claims_div.find_elements(By.CSS_SELECTOR, ".claim-text.patent-text")
    # print(pubnum.text)
    # print(title.text)
    # print(abstract.text)
    print("-----")
    #Inventors and assignee
    result = {}
    try:
        people_div = driver.find_element(By.CSS_SELECTOR, ".important-people")
        inventors_st_mod = people_div.find_elements(By.CSS_SELECTOR, "state-modifier a")
        inventors = []
        for inventor in inventors_st_mod:
            inventors.append(inventor.text)
        result['inventors'] = inventors
    except:
        result['inventors'] = []
    #print(result['inventors'])

    footer_div = driver.find_element(By.CSS_SELECTOR, "div.footer.patent-result")

    h3s_element = footer_div.find_elements(By.CSS_SELECTOR, "h3.patent-result")
    citations_idx = -1
    citedby_idx = -1
    print(len(h3s_element))
    for idx, h3 in enumerate(h3s_element):
        print(h3.text)
        h3_text = h3.text
        if h3_text.startswith("Cited By"):
            citedby_idx = idx
        if h3_text.startswith("Patent Citations"):
            citations_idx = idx
    
    print("C1, c2 => ", citations_idx, citedby_idx)
            

    # pt_citations_element = footer_div.find_elements(By.CSS_SELECTOR, "h3#patentCitations")
    # pt_citations_text = pt_citations_element.text
    # match = re.search(r'\((\d+)\)', pt_citations_text)
    # total_citations = match.group(1)
    # print(total_citations, " - Total Citations")

    # responsive_tbl_divs = footer_div.find_elements(By.CSS_SELECTOR, "div.responsive-table.patent-result")
    # tr_divs = responsive_tbl_divs[0].find_elements(By.CSS_SELECTOR, "div.tr.patent-result")
    # tr_divs = tr_divs[1:]
    # citations_own = []
    # own_citations_full = False
    # citations_ftf = []
    # for item in tr_divs:
    #     #print(item.text)
    #     patent_id = item.text.split(" ")[0]
    #     if patent_id == 'Family':
    #         own_citations_full = True
    #         continue
    #     if own_citations_full:
    #         citations_ftf.append(patent_id)
    #     else:
    #         citations_own.append(patent_id)

    # print(citations_own)
    # print(len(citations_own))
    # print(len(citations_ftf))

    # pt_cited_by_element = footer_div.find_element(By.CSS_SELECTOR, "h3#citedBy")
    # pt_citedby_text = pt_cited_by_element.text
    # match = re.search(r'\((\d+)\)', pt_citedby_text)
    # total_citedby = match.group(1)
    # print(total_citedby, " - Total Cited By")

    # tr_divs = responsive_tbl_divs[1].find_elements(By.CSS_SELECTOR, "div.tr.patent-result")
    # tr_divs = tr_divs[1:]
    # print("length - ", len(tr_divs))
    # citedby_own = []
    # own_citedby_full = False
    # citedby_ftf = []
    # for item in tr_divs:
    #     #print(item.text)
    #     patent_id = item.text.split(" ")[0]
    #     if patent_id == 'Family':
    #         own_citedby_full = True
    #         continue
    #     if own_citedby_full:
    #         citedby_ftf.append(patent_id)
    #     else:
    #         citedby_own.append(patent_id)
    # print(len(citedby_own))
    # print(len(citedby_ftf))

    print("--------")
#test_selenium()

In [8]:
#test_selenium("https://patents.google.com/patent/US11604547B2/en")

In [9]:
#url2 = df['result link'][2]
# count = 0
# for url in df['result link']:
#     test_selenium(url)
#     count += 1
#     if count == 10:
#         break
#test_selenium("https://patents.google.com/patent/US11777957B2/en")

In [10]:
#Function to call fetch list of patents belonging to a specific classification
#Do not run this as repeated downloads lead to rate limited blocking from google
def fetch_category_csv(classification):
    url_encoded_classification = classification.replace("/", "-")
    url = f"https://patents.google.com/?q={classification}&country=US"
    download_url = f"https://patents.google.com/xhr/query?url=q={classification}&country=US&exp=&download=true"
    print(f"Url: {url}")
    print(f"Downlod url: {download_url}")
    try:
        urlretrieve(download_url, f"{url_encoded_classification}.csv")
    except Exception as e:
        print(e)


In [11]:
def fetch_100_categories():
    count = 0
    for idx, item in df.iterrows():
        classification = item['classification']
        if classification.startswith('G06N'):
            count +=1
            print("Fetching class/subclass: ", item['classification'])
            fetch_category_csv(classification)
        if count == 100:
            break

In [12]:
#fetch list of categories related to Machine Learning/ Artificial Intelligence (G06N)
#Do not run this as repeated downloads lead to rate limited blocking from google
#fetch_100_categories()

In [13]:
dataset_folder = "./dataset"
if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)
    print("Dataset folder created")
else:
    print("Dataset folder already exists")

Dataset folder already exists


In [14]:
def saveToFile(content):
    file_path = f"./dataset/{content['patent_id']}/{content['patent_id']}.json"
    json_data = json.dumps(content, indent=4)
    with open(file_path, 'w') as json_file:
        json_file.write(json_data)
        print(f"File created for {content['patent_id']} at path: {file_path}")

In [15]:
#copy classifications to CSV
global_classifications = []
def save_to_csv():
    global global_classifications
    with open('./scraper_list/classifiction_list.csv', 'a', newline = '') as result_file:
        wr = csv.writer(result_file, dialect='excel')
        for item in global_classifications:
            wr.writerow([item])

In [16]:
def scrape(patent_link, result = {}):
    global global_classifications
    driver = webdriver.Chrome()
    driver.get(patent_link)
    driver.implicitly_wait(5)
    result['patent_link'] = patent_link
    #Inventors and assignee
    try:
        people_div = driver.find_element(By.CSS_SELECTOR, ".important-people")
        inventors_st_mod = people_div.find_elements(By.CSS_SELECTOR, "state-modifier a")
        inventors = []
        for inventor in inventors_st_mod:
            inventors.append(inventor.text)
        result['inventors'] = inventors
    except:
        result['inventors'] = []
    
    #classifications
    try:
        view_more_div = driver.find_element(By.CSS_SELECTOR, "classification-viewer div.more.classification-viewer")
        view_more_div.click()
        classifications = []
        classification_viewer = driver.find_element(By.CSS_SELECTOR, "classification-viewer.patent-result")
        a_tags = classification_viewer.find_elements(By.CSS_SELECTOR, "classification-tree.classification-viewer a")
        for a_tag in a_tags:
            if a_tag.text.strip():
                classifications.append(a_tag.text)
        result['classifications'] = classifications
        global_classifications = list(set(global_classifications + classifications))
    except:
        result['classifications'] = []

    try:
        abstract = driver.find_element(By.CSS_SELECTOR, ".abstract.patent-text")
        result['abstract'] = abstract.text
    except:
        result['abstract'] = ""
    try:
        claims_div = driver.find_element(By.CSS_SELECTOR, ".claims.patent-text")
        claims = claims_div.find_elements(By.CSS_SELECTOR, ".claim-text.patent-text")
        claims_text = ""
        for claim in claims:
            if re.match(r"[0-9]+.", claim.text):
                claims_text += "\n"+claim.text
        result['claims'] = claims_text
    except Exception:
        result['claims'] = ""
    try:
        app_tl_div = driver.find_element(By.CSS_SELECTOR, "application-timeline.patent-result > div.application-timeline")
        app_tl_children = app_tl_div.find_elements(By.CSS_SELECTOR, "div .event.layout.horizontal.application-timeline")
        for child in app_tl_children:
            split_child = child.text.split('\n')
            if split_child[0] == 'Status':
                result['status'] = split_child[1]
    except Exception:
        result['status'] = ""

    try:
        footer_div = driver.find_element(By.CSS_SELECTOR, "div.footer.patent-result")

        h3s_element = footer_div.find_elements(By.CSS_SELECTOR, "h3.patent-result")
        citations_idx = -1
        citedby_idx = -1
        for idx, h3 in enumerate(h3s_element):
            h3_text = h3.text
            if h3_text.startswith("Cited By"):
                citedby_idx = idx
            if h3_text.startswith("Patent Citations"):
                citations_idx = idx
        
        #print("C1, c2 => ", citations_idx, citedby_idx)

        pt_citations_element = footer_div.find_element(By.CSS_SELECTOR, "h3#patentCitations")
        pt_citations_text = pt_citations_element.text
        match = re.search(r'\((\d+)\)', pt_citations_text)
        total_citations = match.group(1)
        #print(total_citations, " - Total Citations")

        responsive_tbl_divs = footer_div.find_elements(By.CSS_SELECTOR, "div.responsive-table.patent-result")
        tr_divs = responsive_tbl_divs[citations_idx].find_elements(By.CSS_SELECTOR, "div.tr.patent-result")
        tr_divs = tr_divs[1:]
        citations_own = []
        own_citations_full = False
        citations_ftf = []
        for item in tr_divs:
            #print(item.text)
            patent_id = item.text.split(" ")[0]
            if patent_id == 'Family':
                own_citations_full = True
                continue
            if own_citations_full:
                citations_ftf.append(patent_id)
            else:
                citations_own.append(patent_id)

        #print(citations_own)
        #print(len(citations_own))
        #print(len(citations_ftf))
        result['citations_own'] = citations_own
        result['citations_ftf'] = citations_ftf

        pt_cited_by_element = footer_div.find_element(By.CSS_SELECTOR, "h3#citedBy")
        pt_citedby_text = pt_cited_by_element.text
        match = re.search(r'\((\d+)\)', pt_citedby_text)
        total_citedby = match.group(1)
        #print(total_citedby, " - Total Cited By")

        tr_divs = responsive_tbl_divs[citedby_idx].find_elements(By.CSS_SELECTOR, "div.tr.patent-result")
        tr_divs = tr_divs[1:]
        #print("length - ", len(tr_divs))
        citedby_own = []
        own_citedby_full = False
        citedby_ftf = []
        for item in tr_divs:
            #print(item.text)
            patent_id = item.text.split(" ")[0]
            if patent_id == 'Family':
                own_citedby_full = True
                continue
            if own_citedby_full:
                citedby_ftf.append(patent_id)
            else:
                citedby_own.append(patent_id)
        #print(len(citedby_own))
        #print(len(citedby_ftf))
        result['citedby_own'] = citedby_own
        result['citedby_ftf'] = citedby_ftf

    except Exception:
        result['citations_own'] = []
        result['citations_ftf'] = []
        result['citedby_own'] = []
        result['citedby_ftf'] = []
    return

In [17]:
def parse_csv(df):
    count = 0
    for index,row in df.iterrows():
        if not row['id'].startswith("US"): continue
        print('Scraping document: ', count+1)
        patent_id = row['id']
        content = {}
        content['patent_id'] = patent_id
        content['title'] = row['title']
        content['assignee'] = row['assignee']
        content['publication_date'] = row['publication date']
        patent_link = row['result link']
        print(f"Fetching patent: {patent_id} from {patent_link}")
        dataset_folder = f"./dataset/{patent_id}"
        scrape(patent_link, content)
        saveToFile(content)
        save_to_csv()
        print(f"File {dataset_folder}/{patent_id}.json created successfully.")
        # if not os.path.exists(dataset_folder+f"/{patent_id}.json"):
        #     scrape(patent_link, content)
        #     saveToFile(content)
        #     save_to_csv()
        #     print(f"File {dataset_folder}/{patent_id}.json created successfully.")
        # else:
        #     print(f"File {dataset_folder}/{patent_id}.json already exists.")
        count += 1
        if count == 100:
            break
    print(count)

In [18]:
df.head()

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link
0,US-11604547-B2,Multipoint touchscreen,Apple Inc.,"Steve Hotelling, Joshua A. Strickon, Brian Q. ...",2004-05-06,2021-01-27,2023-03-14,2023-03-14,https://patents.google.com/patent/US11604547B2/en,https://patentimages.storage.googleapis.com/a1...
1,US-11029838-B2,"Touch screen device, method, and graphical use...",Apple Inc.,"Stephen O. Lemay, Richard Williamson",2006-09-06,2019-12-04,2021-06-08,2021-06-08,https://patents.google.com/patent/US11029838B2/en,https://patentimages.storage.googleapis.com/bf...
2,US-11379541-B2,System and method for adapting a control funct...,Autoconnect Holdings Llc,Christopher P. Ricci,2013-04-15,2021-04-16,2022-07-05,2022-07-05,https://patents.google.com/patent/US11379541B2/en,https://patentimages.storage.googleapis.com/f0...
3,US-11390881-B2,Soybean event MON89788 and methods for detecti...,"Monsanto Technology, Llc","Marianne Malven, Jennifer Rinehart, Nancy Tayl...",2005-05-27,2020-07-01,2022-07-19,2022-07-19,https://patents.google.com/patent/US11390881B2/en,https://patentimages.storage.googleapis.com/3f...
4,US-11423886-B2,Task flow identification based on user intent,Apple Inc.,"Thomas Robert Gruber, Adam John Cheyer, Dag Ki...",2010-01-18,2020-05-20,2022-08-23,2022-08-23,https://patents.google.com/patent/US11423886B2/en,https://patentimages.storage.googleapis.com/6a...


In [98]:
parse_csv(df)

Scraping document:  1
Fetching patent: US-11604547-B2 from https://patents.google.com/patent/US11604547B2/en
File created for US-11604547-B2 at path: ./dataset/US-11604547-B2/US-11604547-B2.json
File ./dataset/US-11604547-B2/US-11604547-B2.json created successfully.
Scraping document:  2
Fetching patent: US-11029838-B2 from https://patents.google.com/patent/US11029838B2/en
File created for US-11029838-B2 at path: ./dataset/US-11029838-B2/US-11029838-B2.json
File ./dataset/US-11029838-B2/US-11029838-B2.json created successfully.
Scraping document:  3
Fetching patent: US-11379541-B2 from https://patents.google.com/patent/US11379541B2/en
File created for US-11379541-B2 at path: ./dataset/US-11379541-B2/US-11379541-B2.json
File ./dataset/US-11379541-B2/US-11379541-B2.json created successfully.
Scraping document:  4
Fetching patent: US-11390881-B2 from https://patents.google.com/patent/US11390881B2/en
File created for US-11390881-B2 at path: ./dataset/US-11390881-B2/US-11390881-B2.json
File

In [19]:
import datetime
now = datetime.datetime.now()
print(now)

2023-11-23 22:40:05.305807


In [20]:
print(now.time())

22:40:05.305807


In [26]:
files = os.listdir("./scraper_list/category_list")
print(len(files), " - total categories")
df = None
print(now.time())
for i in range(38,45):
    print(now.time())
    df = pd.read_csv(f"./scraper_list/category_list/{files[i]}")
    print(files[i])
    df.columns
    parse_csv(df)

55  - total categories
22:40:05.305807
22:40:05.305807
gp-search-20231106-010256.csv
Scraping document:  1
Fetching patent: US-2019258943-A1 from https://patents.google.com/patent/US20190258943A1/en
File created for US-2019258943-A1 at path: ./dataset/US-2019258943-A1/US-2019258943-A1.json
File ./dataset/US-2019258943-A1/US-2019258943-A1.json created successfully.
Scraping document:  2
Fetching patent: US-11348022-B2 from https://patents.google.com/patent/US11348022B2/en
File created for US-11348022-B2 at path: ./dataset/US-11348022-B2/US-11348022-B2.json
File ./dataset/US-11348022-B2/US-11348022-B2.json created successfully.
Scraping document:  3
Fetching patent: US-11113614-B2 from https://patents.google.com/patent/US11113614B2/en
File created for US-11113614-B2 at path: ./dataset/US-11113614-B2/US-11113614-B2.json
File ./dataset/US-11113614-B2/US-11113614-B2.json created successfully.
Scraping document:  4
Fetching patent: US-11010671-B2 from https://patents.google.com/patent/US1101

FileNotFoundError: [Errno 2] No such file or directory: './dataset/US-11276004-B2/US-11276004-B2.json'